In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from pandas.tseries.offsets import DateOffset

In [2]:
df_sales = pd.read_csv('Sales_Data05.csv')
df_member = pd.read_csv('Member_Data.csv',encoding='cp949')
df_product = pd.read_csv('Product_Data.csv')

In [4]:
# 1. 데이터 전처리 및 고객별 피처 엔지니어링
# =============================================================================

# (1) 날짜 형식 변환: 주문일시가 문자열이라면 datetime으로 변환
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'], errors='coerce')
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'], errors='coerce')
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce')
df_sales['배송기간'] = (df_sales['배송완료일'] - df_sales['주문일시']).dt.days 

#배송 지연 여부 생성: 배송기간이 2일 초과이면 1, 아니면 0
threshold_days = 2  # 이 값을 변경하면 기준 조정 가능
df_sales['배송지연여부'] = (df_sales['배송기간'] > threshold_days).astype(int)

print(df_sales[['주문일시', '배송완료일', '배송기간', '배송지연여부']].head())

# (2) 구독여부 전처리: 문자열인 경우 소문자화 및 매핑 (예: 'true', 'yes' -> True)
df_member['구독여부'] = df_member['구독여부'].astype(str).str.strip().str.lower()
mapping_dict = {'true': True, 'false': False, 'yes': True, 'no': False}
df_member['구독여부'] = df_member['구독여부'].map(mapping_dict)

# -- 고객별 구매 내역, 배송지연 빈도 계산 --
# (a) 고객별 총 주문 건수, 총 구매 금액, 배송지연 주문 수 계산
customer_orders = df_sales.groupby('회원번호').agg(
    total_orders = ('회원번호', 'count'),
    total_purchase_amount = ('구매금액', 'sum'),
    shipping_delay_count = ('배송지연여부', 'sum')   # 배송지연여부가 1이면 배송 지연으로 간주
).reset_index()

customer_orders['shipping_delay_freq'] = customer_orders['shipping_delay_count'] / customer_orders['total_orders']

# (b) 고객별 평균 구매 주기 계산
def compute_avg_cycle(dates):
    dates = dates.sort_values()
    if len(dates) < 2:
        return np.nan
    # 주문일시 간 차이를 일(day) 단위로 계산
    diffs = dates.diff().dropna().dt.days
    return diffs.mean()

purchase_cycle = df_sales.groupby('회원번호')['주문일시'].apply(compute_avg_cycle).reset_index(name='avg_purchase_cycle')

# (c) 고객별 마지막 주문일 계산
last_order = df_sales.groupby('회원번호')['주문일시'].max().reset_index(name='last_order_date')

# (d) 고객별 피처 병합
customer_features = customer_orders.merge(purchase_cycle, on='회원번호', how='left')
customer_features = customer_features.merge(last_order, on='회원번호', how='left')

# (e) 회원 정보(구독 여부) 병합
customer_features = customer_features.merge(df_member[['회원번호', '구독여부']], on='회원번호', how='left')

# (f) 결측치 처리: 평균 구매 주기가 NaN인 경우 전체 중앙값으로 대체
customer_features['avg_purchase_cycle'] = customer_features['avg_purchase_cycle'].fillna(customer_features['avg_purchase_cycle'].median())

        주문일시      배송완료일  배송기간  배송지연여부
0 2021-01-02        NaT   NaN       0
1 2021-01-02 2021-01-02   0.0       0
2 2021-01-02 2021-01-03   1.0       0
3 2021-01-02 2021-01-03   1.0       0
4 2021-01-02        NaT   NaN       0


In [5]:
# 2. 라벨 생성: 유령/이탈 고객 여부 (목표 변수)
# =============================================================================

# 유령(이탈) 고객 정의: 탈퇴하지 않은 고객 중, 최근 3개월 이내에 주문 기록이 없는 고객
# 기준 날짜: df_sales의 최대 주문일
reference_date = df_sales['주문일시'].max()
threshold_date = reference_date - DateOffset(months=3)

# 고객별 마지막 주문일이 threshold_date보다 이전이면 유령/이탈 고객으로 라벨 1, 아니면 0
customer_features['is_ghost'] = (customer_features['last_order_date'] < threshold_date).astype(int)

In [8]:
# 3. 모델 입력 변수 구성
# =============================================================================

# 구독 여부는 Boolean이므로 int형(0,1)으로 변환
# NaN 값을 0으로 채우고, int형으로 변환 (구독하지 않은 경우를 0으로 간주)
customer_features['구독여부'] = customer_features['구독여부'].fillna(0).astype(int)


# 선택할 피처: 구매 내역, 구매 주기, 배송지연 빈도, 구독 여부
X = customer_features[['total_orders', 'total_purchase_amount', 'avg_purchase_cycle', 'shipping_delay_freq', '구독여부']]
y = customer_features['is_ghost']

print("고객 피처 예시:")
print(customer_features.head())

고객 피처 예시:
       회원번호  total_orders  total_purchase_amount  shipping_delay_count  \
0  18764160            36                 811902                     5   
1  18792000             1                   9955                     0   
2  18942336            81                1418288                     7   
3  18949760             2                  35130                     0   
4  19391488             1                   3159                     0   

   shipping_delay_freq  avg_purchase_cycle last_order_date  구독여부  is_ghost  
0             0.138889            7.885714      2021-10-06     0         0  
1             0.000000            4.555556      2021-04-05     0         1  
2             0.086420            3.550000      2021-10-20     0         0  
3             0.000000            0.000000      2021-09-30     0         0  
4             0.000000            4.555556      2021-05-19     0         1  


In [9]:
# 4. 모델 학습: 로지스틱 회귀를 통한 확률 예측 (또는 분류)
# =============================================================================

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 (클래스 불균형 문제에 대응하여 class_weight='balanced' 적용)
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
log_reg.fit(X_train, y_train)

# 예측 확률 및 클래스
y_pred_proba = log_reg.predict_proba(X_test)[:, 1]  # ghost 고객일 확률
y_pred = log_reg.predict(X_test)


In [10]:
# 5. 모델 평가
# =============================================================================

print("\n----- Ghost/Churn Prediction Model Evaluation -----")
print("혼동 행렬:")
print(confusion_matrix(y_test, y_pred))
print("\n분류 리포트:")
print(classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_proba))

# 예시: 특정 고객에 대해 예측 확률 확인
example_customer = X_test.iloc[0]
example_probability = log_reg.predict_proba([example_customer])[0, 1]
print("\n예시 고객의 ghost/churn 될 확률: {:.2f}%".format(example_probability * 100))


----- Ghost/Churn Prediction Model Evaluation -----
혼동 행렬:
[[1932  747]
 [ 125  958]]

분류 리포트:
              precision    recall  f1-score   support

           0       0.94      0.72      0.82      2679
           1       0.56      0.88      0.69      1083

    accuracy                           0.77      3762
   macro avg       0.75      0.80      0.75      3762
weighted avg       0.83      0.77      0.78      3762

ROC AUC Score: 0.8641478797679845

예시 고객의 ghost/churn 될 확률: 3.90%


C:\Users\sylvi\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
